<a href="https://colab.research.google.com/github/fahadrafiq94/Machine-Learning/blob/main/07_Mile_stone_project_TO_DO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-b2557fe0-4530-569c-5b7f-b9718f56872a)


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-09-20 09:02:28--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-09-20 09:02:28 (65.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import create_tensorboard_callback , compare_historys , plot_loss_curves

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [ ]:
(train_data , test_data)  ,ds_info = tfds.load("food101",
                                               split =["train" , "validation"],
                                               shuffle_files =True,
                                               as_supervised =True,
                                               with_info =True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteFXHWED/food101-train.tfrecord


  0%|          | 0/75750 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteFXHWED/food101-validation.tfrecord


  0%|          | 0/25250 [00:00<?, ? examples/s]

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [ ]:
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [ ]:
class_names = ds_info.features["label"].names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [ ]:
one_train_data = train_data.take(1)

In [ ]:
one_train_data

<TakeDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [ ]:
for image , label in one_train_data:
  print(f"""
  image_shape : {image.shape}
  image_dtype : {image.dtype}
  target_class :{label}
  class_name : {class_names[label.numpy()]}
  """)


  image_shape : (512, 512, 3)
  image_dtype : <dtype: 'uint8'>
  target_class :23
  class_name : churros
  


In [ ]:
import tensorflow as tf

In [ ]:
def preprocess_img(image , label  ,  img_shape=224):
  image  = tf.image.resize(image , [img_shape , img_shape])
  return tf.cast(image ,dtype=tf.float32) , label

In [ ]:
preprocessed_img = preprocess_img(image , label )[0]
preprocessed_img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 93.      ,  34.      ,  40.      ],
        [ 94.      ,  35.      ,  41.      ],
        [ 95.      ,  36.      ,  42.      ],
        ...,
        [ 55.      ,  15.      ,  24.      ],
        [ 52.      ,  15.      ,  23.      ],
        [ 52.      ,  15.      ,  23.      ]],

       [[ 93.92857 ,  34.928574,  40.928574],
        [ 93.07143 ,  34.071426,  40.071426],
        [ 94.      ,  35.      ,  41.      ],
        ...,
        [ 55.      ,  15.      ,  24.      ],
        [ 54.      ,  14.      ,  23.      ],
        [ 52.      ,  15.      ,  23.      ]],

       [[ 93.928566,  35.357143,  41.57143 ],
        [ 91.64285 ,  33.071426,  39.285713],
        [ 91.78571 ,  33.214287,  39.428574],
        ...,
        [ 54.      ,  14.      ,  23.      ],
        [ 55.      ,  15.      ,  24.      ],
        [ 55.      ,  15.      ,  24.      ]],

       ...,

       [[108.01023 ,  31.010233,  49.01023 ],
        [108.

In [ ]:
train_data = train_data.map(map_func=preprocess_img , num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = test_data.map(map_func=preprocess_img , num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_data , test_data

(<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>,
 <PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>)

In [ ]:
checkpoint_path = "checkpoint_path/ck.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      verbose=0)

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla K80, compute capability 3.7
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla K80, compute capability 3.7
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:
mixed_precision.global_policy()

<Policy "mixed_float16">

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

image_shape =(224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable =False

inputs = layers.Input(shape=image_shape , name ="input_layer")
x = base_model(inputs , training= False)
x = layers.GlobalAveragePooling2D(name="globalaverage_pooling2D")(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax" , dtype=tf.float32, name="output_layer")(x)

model = tf.keras.Model(inputs , outputs)

16719872/16705208 [==============================] - 0s 0us/step


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer =tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
globalaverage_pooling2D (Glo (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 101)               129381    
_________________________________________________________________
output_layer (Activation)    (None, 101)               0         
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
for layer in model.layers:
  print(layer.name ,  layer.dtype , layer.trainable)

input_layer float32 True
efficientnetb0 float32 False
globalaverage_pooling2D float32 True
dense float32 True
output_layer float32 True


In [ ]:
for layer in model.layers[1].layers[:10]:
  print(layer.name ,  layer.dtype , layer.trainable , layer.dtype_policy)

input_1 float32 False <Policy "float32">
rescaling float32 False <Policy "mixed_float16">
normalization float32 False <Policy "mixed_float16">
stem_conv_pad float32 False <Policy "mixed_float16">
stem_conv float32 False <Policy "mixed_float16">
stem_bn float32 False <Policy "mixed_float16">
stem_activation float32 False <Policy "mixed_float16">
block1a_dwconv float32 False <Policy "mixed_float16">
block1a_bn float32 False <Policy "mixed_float16">
block1a_activation float32 False <Policy "mixed_float16">


### TO DO from here.


#fit the feature extraction model for 3 epochs on 15% of validation data

In [ ]:
history_model_feature_extraction = model.fit(train_data,
                                             epochs=3,
                                             steps_per_epoch=len(train_data),
                                             validation_data=test_data,
                                             validation_steps=int(0.15*len(test_data)),
                                             callbacks=[create_tensorboard_callback("tensorboard" ,
                                                                                    "EfficientNetb0_fetaure_extraction"),
                                                        model_checkpoint])

Saving TensorBoard log files to: tensorboard/EfficientNetb0_fetaure_extraction/20210905-025941
Epoch 1/3


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


2368/2368 [==============================] - 317s 115ms/step - loss: 1.8198 - accuracy: 0.5573 - val_loss: 1.2225 - val_accuracy: 0.6790
Epoch 2/3
2368/2368 [==============================] - 254s 106ms/step - loss: 1.2934 - accuracy: 0.6659 - val_loss: 1.1219 - val_accuracy: 0.6978
Epoch 3/3
2368/2368 [==============================] - 253s 106ms/step - loss: 1.1436 - accuracy: 0.7012 - val_loss: 1.0870 - val_accuracy: 0.7013


In [ ]:
results_model_feature_extraction = model.evaluate(test_data)

790/790 [==============================] - 84s 107ms/step - loss: 1.0865 - accuracy: 0.7055


In [ ]:
model.save("/content/drive/MyDrive/model_feature_extraction")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/model_feature_extraction/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model_feature_extraction/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/model_feature_extraction")

In [ ]:
results_loaded_model_feature_extraction =loaded_model.evaluate(test_data)

790/790 [==============================] - 82s 101ms/step - loss: 1.0865 - accuracy: 0.7055


In [ ]:
for layer in base_model.layers:
  print(layer.name , layer.dtype_policy)

input_1 <Policy "float32">
rescaling <Policy "mixed_float16">
normalization <Policy "mixed_float16">
stem_conv_pad <Policy "mixed_float16">
stem_conv <Policy "mixed_float16">
stem_bn <Policy "mixed_float16">
stem_activation <Policy "mixed_float16">
block1a_dwconv <Policy "mixed_float16">
block1a_bn <Policy "mixed_float16">
block1a_activation <Policy "mixed_float16">
block1a_se_squeeze <Policy "mixed_float16">
block1a_se_reshape <Policy "mixed_float16">
block1a_se_reduce <Policy "mixed_float16">
block1a_se_expand <Policy "mixed_float16">
block1a_se_excite <Policy "mixed_float16">
block1a_project_conv <Policy "mixed_float16">
block1a_project_bn <Policy "mixed_float16">
block2a_expand_conv <Policy "mixed_float16">
block2a_expand_bn <Policy "mixed_float16">
block2a_expand_activation <Policy "mixed_float16">
block2a_dwconv_pad <Policy "mixed_float16">
block2a_dwconv <Policy "mixed_float16">
block2a_bn <Policy "mixed_float16">
block2a_activation <Policy "mixed_float16">
block2a_se_squeeze <P

In [ ]:
results_model_feature_extraction[1] ==results_loaded_model_feature_extraction[1]

True

In [ ]:
# Download the saved model from Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip 

--2021-09-05 03:17:40--  https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.219.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.219.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16976857 (16M) [application/zip]
Saving to: ‘07_efficientnetb0_feature_extract_model_mixed_precision.zip’

07_efficientnetb0_f 100%[===================>]  16.19M  52.5MB/s    in 0.3s    

2021-09-05 03:17:41 (52.5 MB/s) - ‘07_efficientnetb0_feature_extract_model_mixed_precision.zip’ saved [16976857/16976857]



In [ ]:
# Unzip the SavedModel downloaded from Google Stroage
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

Archive:  07_efficientnetb0_feature_extract_model_mixed_precision.zip
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/variables.data-00000-of-00001  
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/variables.index  
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/saved_model.pb  
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/assets/


In [ ]:
downloaded_gs_model = tf.keras.models.load_model("/content/downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:
results_downloaded_gs_model =downloaded_gs_model.evaluate(test_data)

790/790 [==============================] - 81s 99ms/step - loss: 1.0881 - accuracy: 0.7067


In [ ]:
downloaded_gs_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layer (GlobalAverage (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 101)               129381    
_________________________________________________________________
softmax_float32 (Activation) (None, 101)               0         
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
for layer in downloaded_gs_model.layers[1].layers:
  layer.trainable=True

In [ ]:
for layer in downloaded_gs_model.layers[1].layers:
  print(layer.name , layer.trainable )

input_1 True
rescaling True
normalization True
stem_conv_pad True
stem_conv True
stem_bn True
stem_activation True
block1a_dwconv True
block1a_bn True
block1a_activation True
block1a_se_squeeze True
block1a_se_reshape True
block1a_se_reduce True
block1a_se_expand True
block1a_se_excite True
block1a_project_conv True
block1a_project_bn True
block2a_expand_conv True
block2a_expand_bn True
block2a_expand_activation True
block2a_dwconv_pad True
block2a_dwconv True
block2a_bn True
block2a_activation True
block2a_se_squeeze True
block2a_se_reshape True
block2a_se_reduce True
block2a_se_expand True
block2a_se_excite True
block2a_project_conv True
block2a_project_bn True
block2b_expand_conv True
block2b_expand_bn True
block2b_expand_activation True
block2b_dwconv True
block2b_bn True
block2b_activation True
block2b_se_squeeze True
block2b_se_reshape True
block2b_se_reduce True
block2b_se_expand True
block2b_se_excite True
block2b_project_conv True
block2b_project_bn True
block2b_drop True
bloc

In [ ]:
for layer in downloaded_gs_model.layers[1].layers:
  print(layer.name , layer.trainable  ,layer.dtype_policy)

input_1 True <Policy "float32">
rescaling True <Policy "mixed_float16">
normalization True <Policy "float32">
stem_conv_pad True <Policy "mixed_float16">
stem_conv True <Policy "mixed_float16">
stem_bn True <Policy "mixed_float16">
stem_activation True <Policy "mixed_float16">
block1a_dwconv True <Policy "mixed_float16">
block1a_bn True <Policy "mixed_float16">
block1a_activation True <Policy "mixed_float16">
block1a_se_squeeze True <Policy "mixed_float16">
block1a_se_reshape True <Policy "mixed_float16">
block1a_se_reduce True <Policy "mixed_float16">
block1a_se_expand True <Policy "mixed_float16">
block1a_se_excite True <Policy "mixed_float16">
block1a_project_conv True <Policy "mixed_float16">
block1a_project_bn True <Policy "mixed_float16">
block2a_expand_conv True <Policy "mixed_float16">
block2a_expand_bn True <Policy "mixed_float16">
block2a_expand_activation True <Policy "mixed_float16">
block2a_dwconv_pad True <Policy "mixed_float16">
block2a_dwconv True <Policy "mixed_float16

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                           patience=3)
checkpoint_path = "checkpoint_path/fine_tune/ck.ckpt"
model_modelpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_loss",
                                                      save_best_model=True)

In [ ]:
downloaded_gs_model.compile(loss="sparse_categorical_crossentropy",
                            optimizer=tf.keras.optimizers.Adam(1e-4),
                            metrics=["accuracy"])

In [ ]:
history_model_fine_tune = downloaded_gs_model.fit(train_data,
                                                  epochs=100,
                                                  steps_per_epoch=len(train_data),
                                                  validation_data=test_data,
                                                  validation_steps=int(0.15*len(test_data)),
                                                  callbacks = [create_tensorboard_callback("tensorboard" ,
                                                                                           "Efficientb0_fine_tune"),
                                                               model_checkpoint,
                                                               early_stopping_callback])

Saving TensorBoard log files to: tensorboard/Efficientb0_fine_tune/20210905-031943
Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


2368/2368 [==============================] - 270s 110ms/step - loss: 0.9718 - accuracy: 0.7521 - val_loss: 1.0449 - val_accuracy: 0.7217
Epoch 2/100
2368/2368 [==============================] - 259s 108ms/step - loss: 0.9549 - accuracy: 0.7568 - val_loss: 1.0440 - val_accuracy: 0.7206
Epoch 3/100
2368/2368 [==============================] - 258s 108ms/step - loss: 0.9440 - accuracy: 0.7597 - val_loss: 1.0468 - val_accuracy: 0.7201
Epoch 4/100
2368/2368 [==============================] - 262s 110ms/step - loss: 0.9345 - accuracy: 0.7619 - val_loss: 1.0451 - val_accuracy: 0.7235
Epoch 5/100
2368/2368 [==============================] - 272s 114ms/step - loss: 0.9257 - accuracy: 0.7652 - val_loss: 1.0441 - val_accuracy: 0.7233


In [ ]:
results_downloaded_gs_model = downloaded_gs_model.evaluate(test_data)

790/790 [==============================] - 80s 101ms/step - loss: 1.0412 - accuracy: 0.7202


In [ ]:
downloaded_gs_model.save("/content/drive/MyDrive/downloaded_GS_Fine_Tuned_Model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/downloaded_GS_Fine_Tuned_Model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/downloaded_GS_Fine_Tuned_Model/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
loaded_download_gs_model = tf.keras.models.load_model("/content/drive/MyDrive/downloaded_GS_Fine_Tuned_Model")

In [ ]:
results_loaded_downloaded_gs_model = loaded_download_gs_model.evaluate(test_data)

790/790 [==============================] - 81s 100ms/step - loss: 1.0412 - accuracy: 0.7202


In [ ]:
base_model.trainable = True 
for layer in base_model.layers:
  print(layer.name , layer.trainable)

input_1 True
rescaling True
normalization True
stem_conv_pad True
stem_conv True
stem_bn True
stem_activation True
block1a_dwconv True
block1a_bn True
block1a_activation True
block1a_se_squeeze True
block1a_se_reshape True
block1a_se_reduce True
block1a_se_expand True
block1a_se_excite True
block1a_project_conv True
block1a_project_bn True
block2a_expand_conv True
block2a_expand_bn True
block2a_expand_activation True
block2a_dwconv_pad True
block2a_dwconv True
block2a_bn True
block2a_activation True
block2a_se_squeeze True
block2a_se_reshape True
block2a_se_reduce True
block2a_se_expand True
block2a_se_excite True
block2a_project_conv True
block2a_project_bn True
block2b_expand_conv True
block2b_expand_bn True
block2b_expand_activation True
block2b_dwconv True
block2b_bn True
block2b_activation True
block2b_se_squeeze True
block2b_se_reshape True
block2b_se_reduce True
block2b_se_expand True
block2b_se_excite True
block2b_project_conv True
block2b_project_bn True
block2b_drop True
bloc

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

In [ ]:
results_model_fine_tune = model.fit(train_data,
                                    epochs=100,
                                    steps_per_epoch=len(train_data),
                                    validation_data=test_data,
                                    validation_steps=int(0.15 *len(test_data)),
                                    callbacks=[create_tensorboard_callback("tensorboard",
                                                                           "efficientNetb0_fine_tune_manual"),
                                               model_checkpoint,
                                               early_stopping_callback])

Saving TensorBoard log files to: tensorboard/efficientNetb0_fine_tune_manual/20210905-041610
Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


2368/2368 [==============================] - 1191s 489ms/step - loss: 0.9238 - accuracy: 0.7523 - val_loss: 0.8202 - val_accuracy: 0.7648
Epoch 2/100
2368/2368 [==============================] - 1141s 481ms/step - loss: 0.5793 - accuracy: 0.8389 - val_loss: 0.8219 - val_accuracy: 0.7701
Epoch 3/100
2368/2368 [==============================] - 1145s 483ms/step - loss: 0.3300 - accuracy: 0.9061 - val_loss: 0.8570 - val_accuracy: 0.7884
Epoch 4/100
2368/2368 [==============================] - 1155s 487ms/step - loss: 0.1754 - accuracy: 0.9477 - val_loss: 1.0306 - val_accuracy: 0.7762


In [ ]:
result_model_fine_tune_manual = model.evaluate(test_data)

790/790 [==============================] - 78s 98ms/step - loss: 1.0447 - accuracy: 0.7692


In [ ]:
model.save("/content/drive/MyDrive/model_fine_tune")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/model_fine_tune/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model_fine_tune/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
!tensorboard dev upload --logdir ./tensorboard \
--name "Food Vision Big TM" \
--description "a series of different expriment using feature extraction and fine tuning" \
--one_shot

2021-09-05 05:47:35.320242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-05 05:47:35.329625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-05 05:47:35.330285: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/aXW0ArxaSdmcbOZhoJfWtQ/

[2021-09-05T05:47:35] Started scanning logdir.
[2021-09-05T05:47:38] Total uploaded: 72 scalars, 0 tensors, 3 binary objects (2.7 MB)
[2021-09-05T05:47:38] Done scanning logdir.


Done. View your TensorBoard at https://tensorboar

Tensorboard Link:https://tensorboard.dev/experiment/aXW0ArxaSdmcbOZhoJfWtQ/

In [ ]:
from tensorflow.keras.layers.experimental  import preprocessing
data_augmentation= tf.keras.Sequential([
     preprocessing.RandomFlip("horizontal"),
     preprocessing.RandomRotation(0.2),
     preprocessing.RandomHeight(0.2),
     preprocessing.RandomWidth(0.2),
     preprocessing.RandomZoom(0.2)                                          
])

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

image_shape =(224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable =False

inputs = layers.Input(shape=image_shape , name ="input_layer")
x =data_augmentation(inputs)
x = base_model(x , training= False)
x = layers.GlobalAveragePooling2D(name="globalaverage_pooling2D")(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax" , dtype=tf.float32, name="output_layer")(x)

model = tf.keras.Model(inputs , outputs)

In [ ]:
base_model.trainable= True
for layer in model.layers:
  print(layer.name , layer.trainable)

input_layer True
sequential_1 True
efficientnetb0 True
globalaverage_pooling2D True
dense_1 True
output_layer True


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

In [ ]:
history_model_augmneted_fine_tuned = model.fit(train_data,
                                    epochs=100,
                                    steps_per_epoch=len(train_data),
                                    validation_data=test_data,
                                    validation_steps=int(0.15 *len(test_data)),
                                    callbacks=[create_tensorboard_callback("tensorboard",
                                                                           "efficientNetb0_fine_tune_augmented"),
                                               model_checkpoint,
                                               early_stopping_callback])

In [ ]:
import tensorflow as tf 

In [ ]:
model_loaded = tf.keras.models.load_model("/content/drive/MyDrive/model_fine_tune")

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla K80, compute capability 3.7
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla K80, compute capability 3.7
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:
test_data

<_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [ ]:
reults_model_loaded = model_loaded.evaluate(test_data)

790/790 [==============================] - 122s 105ms/step - loss: 1.0447 - accuracy: 0.7692


In [ ]:
def get_images_from_tfdataset(tfdataset, batched=False):

    images = list(map(lambda x: x[0], tfdataset)) # Get images

    if not batched:
        return tf.concat(images, axis=0) # concat the list of batched labels

    return images
    

In [ ]:
test_images = get_images_from_tfdataset(test_data , batched=False)

In [ ]:
def parse_row(ds):
  return ds[]

In [ ]:
test_dataset_images = parse_row(test_data)

In [ ]:
preds_prob = model_loaded.predict(test_data  ,verbose=1)

790/790 [==============================] - 67s 83ms/step


In [ ]:
preds_prob.shape

(25250, 101)

In [ ]:
pred_classes = preds_prob.argmax(axis=1)
pred_classes[:10]

array([ 1, 75, 74, 61, 54, 76, 90, 34,  4, 44])

In [ ]:
for i in range(1):
  preds_prob[i] ,len(preds_prob[i]) ,sum(preds_prob[i])

[3.02719898e-11 3.06196800e-13 1.39598897e-15 3.22729074e-14
 1.55265667e-09 2.81825669e-06 1.31061097e-06 2.53846044e-13
 2.63007584e-11 8.75948966e-11 8.27589993e-12 5.48980620e-14
 1.95745079e-14 4.20918014e-14 1.03429232e-09 1.61354369e-13
 5.36583777e-10 5.23840953e-14 1.37227881e-12 2.31129487e-13
 1.33005824e-12 4.02362943e-02 3.58408070e-05 1.12304896e-07
 6.58426547e-13 1.44854995e-16 8.51558684e-17 1.92420437e-11
 2.63403767e-14 6.72014551e-07 1.70782854e-12 8.13533951e-09
 3.88699975e-11 2.14983908e-14 2.16214985e-15 1.95745079e-14
 6.21140847e-11 1.10099851e-09 3.34880299e-15 5.34332344e-12
 4.13769019e-11 8.77268500e-14 9.52550365e-12 8.37095489e-14
 4.16765325e-15 8.62368649e-11 1.01825715e-09 6.50949988e-11
 7.58986043e-16 9.52550365e-12 6.28275101e-13 2.06871492e-10
 5.84735871e-10 9.63490897e-14 1.33968981e-16 3.46966433e-13
 8.91083472e-14 8.35836400e-11 4.56351223e-09 2.93848130e-14
 1.01398464e-11 1.91901707e-16 7.85195589e-11 5.00151087e-10
 2.50963590e-11 1.483787

In [ ]:
test_data

<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>